In [ ]:
### RUN ONLY ON KAGGLE 

In [ ]:
!pip install torch==1.10.2+cu102 torchvision==0.11.3+cu102 torchaudio===0.10.2+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html


In [ ]:
!pip install transformers tensorboardX
!pip install sentencepiece
import os
from os.path import join, abspath, dirname
import requests
import json
import numpy as np
from itertools import islice
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel, RobertaModel
from transformers import XLNetTokenizer, XLNetModel
from torch.utils import data as torch_data
from torch.nn.utils.rnn import pad_sequence
from torch.multiprocessing import set_start_method
from datetime import datetime
from tqdm import trange, tqdm
from tensorboardX import SummaryWriter

     |████████████████████████████████| 3.8 MB 7.4 MB/s 
     |████████████████████████████████| 125 kB 67.8 MB/s 
     |████████████████████████████████| 6.5 MB 41.7 MB/s 
     |████████████████████████████████| 895 kB 54.4 MB/s 
     |████████████████████████████████| 67 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 41.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 9.6 MB/s 


In [ ]:
torch.cuda.is_available()

In [ ]:
import shutil

In [ ]:
!pip install --upgrade --no-cache-dir gdown
!gdown --id 1IcnIsxRCl4k-fKD_sAYqeiHsMAyiZ-jA
# https://drive.google.com/file/d/1p6ZPMmRcqE_jI71q-TCzavaegnnN3ofO/view?usp=sharing
# !pip install gdown

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1IcnIsxRCl4k-fKD_sAYqeiHsMAyiZ-jA
To: /content/checkpoints.zip
100% 1.08G/1.08G [00:32<00:00, 33.4MB/s]


In [ ]:
import gdown

In [ ]:
file_paths = {
    "path_to_raw_data": "./processed/random_data.json",
    "path_to_target_vocab": "./processed/target_vocab.json",
    "path_to_char_vocab": "./processed/random_char_vocab.json",
    "path_to_glove_embed": "./processed/random_glove_embed.npy",
    "path_to_glove_vocab": "./processed/random_glove_vocab.json",
    "path_to_pos_vocab": "./processed/random_pos_vocab.json"#,
    #"path_to_xlnet_vocab": "./processed/random_xlnet_vocab.json",
   # "path_to_xlnet_indices": "./processed/random_xlnet_indices.json"
}


In [ ]:
def get_processed_data():
    #https://drive.google.com/file/d/1UP6BCzUlXSJOnwawK_BtuAcP_I5pn9yQ/view?usp=sharing
    url = 'https://drive.google.com/uc?id=1p6ZPMmRcqE_jI71q-TCzavaegnnN3ofO'
    # url = 'https://drive.google.com/uc?id=1OER5wepwviNUopmgUb08EzcTL3qCb5Ps'
    output = 'processed.zip'
    gdown.download(url, output, quiet=False)
    
#     with zipfile.Zipfile('processed.zip', 'r') as zip_ref:
#         zip.ref.extractall('./processed/')
    shutil.unpack_archive('processed.zip', './')
        
    return

In [ ]:
# torch.device('hpu')

In [ ]:
#get_processed_data()

In [ ]:
def get_raw_data():
    url = "https://raw.githubusercontent.com/hslh/magpie-corpus/master/MAGPIE_filtered_split_random.jsonl"
    r = requests.get(url, allow_redirects=True)

    open('meta/meta_data_magpie_random.json', 'wb').write(r.content)
    

In [ ]:
def get_checkpoint():
    url = 'https://drive.google.com/uc?id=1IcnIsxRCl4k-fKD_sAYqeiHsMAyiZ-jA'
    #https://drive.google.com/file/d/1IcnIsxRCl4k-fKD_sAYqeiHsMAyiZ-jA/view?usp=sharing
    output = 'checkpoints.zip'
    gdown.download(url, output, quiet=False)
    
#     with zipfile.Zipfile('processed.zip', 'r') as zip_ref:
#         zip.ref.extractall('./processed/')
    shutil.unpack_archive('checkpoints.zip', './')
    return

In [ ]:
def generate_folders():
    ROOT= os.getcwd()
    try:
        get_checkpoint()
        if not os.path.isdir(os.path.join(ROOT, "/checkpoints")):
            os.mkdir(os.path.join(ROOT, "checkpoints"))
        
    except FileExistsError:
        pass
    
    try:
        if not os.path.isdir(os.path.join(ROOT, "/meta")):
            os.mkdir(os.path.join(ROOT, "meta"))
            get_raw_data()
            
    except FileExistsError:
        try:
            if not os.path.isfile(os.path.join(ROOT, "/meta/meta_data_magpie_random.json")):
                get_raw_data()
        except FileExistsError:
            pass
        
    try:
        get_processed_data()
        
        if not os.path.isdir(os.path.join(ROOT, "/processed")):
            os.mkdir(os.path.join(ROOT, "processed"))
        shutil.unpack_archive('/content/processed.zip',  '/content/')
            
    except FileExistsError:
        shutil.unpack_archive('/content/processed.zip',  '/content/')
        pass
    
    try:
        if not os.path.isdir(os.path.join(ROOT, "/results")):
            os.mkdir(os.path.join(ROOT, "results"))
    except FileExistsError:
        pass
    

In [ ]:
#!mkdir ./tmp

In [ ]:
#!rm -rf /content/processed/*

In [ ]:
# shutil.unpack_archive('./processed.zip',  './tmp/')
get_checkpoint()

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1IcnIsxRCl4k-fKD_sAYqeiHsMAyiZ-jA 



ReadError: ignored

In [ ]:
generate_folders()

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1IcnIsxRCl4k-fKD_sAYqeiHsMAyiZ-jA 



Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1p6ZPMmRcqE_jI71q-TCzavaegnnN3ofO 



ReadError: ignored

## Configuration

In [ ]:
#!rm -rf /content/processed/

In [ ]:
#!ls ./processed

In [ ]:
class Config:
    ROOT = os.getcwd()

    # Settings
    MODE = 'train'
    DATA_NAME = 'magpie'
    SPLIT = 'random'
    MODEL_NAME = 'IdiomDetector_{}_{}'.format(DATA_NAME, SPLIT)
    PATH_TO_META_DATA = file_paths
    SEED = 199
    OUTPUT_ATTN = False

    USE_GPU = True
    CONTINUE_TRAIN=False
    
    USE_TENSORBOARD = True
    VERBOSE = False
    NUM_WORKER = 0

    # Checkpoint management
    PATH_TO_CHECKPOINT = join(ROOT, 'checkpoints/{}_{}.mdl')
    LOAD_CHECKPOINT_TYPE = 'best'

    # Train Parameters
    NUM_EPOCHS = 1
    BATCH_SIZE = 56
    VALID_FREQ = 1
    SAVE_FREQ = 10
    DISPLAY_FREQ = 10
    LEARNING_RATE = 1e-4

    # Inference parameters
    PATH_TO_SAVE_PERFORMANCE = join(ROOT, 'results/{}_inference_performance.json'.format(MODEL_NAME))
    PATH_TO_SAVE_RESULTS = join(ROOT, 'results/{}_{}inference_RESULTS.json'.format(DATA_NAME, SPLIT))
    
    # Data Parameters
    MAX_SEQ_LEN = 512

    START_SYMBOL = '<s>'
    END_SYMBOL = '<e>'
    PAD_SYMBOL = '<PAD>'

#     DEVICE = 'cpu'
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    #Embeddings

    PRETRAINED_BERT_MODEL_NAME = 'bert-base-uncased'
    PRETRAINED_XLNET_MODEL_NAME = 'xlnet-large-cased'
    PRETRAINED_BERT_MODEL_EMBED_DIM = 768
    PRETRAINED_XLNET_MODEL_EMBED_DIM = 1024
    PRETRAINED_LM__MODEL_EMBED_DIM = PRETRAINED_BERT_MODEL_EMBED_DIM # LINEAR MODEL
    PRETRAINED_GLOVE_MODEL_EMBED_DIM = 300
    CHAR_EMBED_DIM = 64
    CHAR_EMBED_DROPOUT_RATE = 0.2
    CHAR_EMBED_CNN_NUM_OUT_CHANNELS = 64
    CHAR_EMBED_CHAR_FILTERS = [[1, 5]]
    POS_EMBED_DIM = 64
    
    # fused embedding dimension
    EMBEDDING_DIM = PRETRAINED_GLOVE_MODEL_EMBED_DIM + CHAR_EMBED_CNN_NUM_OUT_CHANNELS

    # Highway Network
    HIGHWAY_NUM_LAYERS = 2

    # LSTM
    LSTM_HIDDEN_DIM = 256
    LSTM_DROP_RATE = 0.3

    # Mode based parameters
    if MODE != 'train':
        LOAD_CHECKPOINT_TYPE = 'latest'
        CONTINUE_TRAIN = True
        USE_TENSORBOARD = False
        BATCH_SIZE = 1

## Data handler for training and validation data

In [ ]:
class Dataset(torch_data.Dataset):

    def __init__(self, df):
        super(Dataset, self).__init__()
        self.df = df
        self.num_total_seqs = len(self.df)

    def __len__(self):
        return self.num_total_seqs

    def __getitem__(self, index):
        return self.df[index]

In [ ]:
# loads .npy file (binary). It contains Glove embeddings
# of dimension 300.
def load_npy_file(path):
    return np.load(path)

In [ ]:
# creates .npy file
def write_npy_file(path, np_array):
    np.save(path, np_array)
    return

In [ ]:
def load_txt_file(path):
    with open(path, 'r') as f:
        return f.read().splitlines()

In [ ]:
def write_txt_file(path, content):
    with open(path, 'w') as f:
        for line in content:
            f.write(line)
            f.write('\n')
        f.close()
    return

In [ ]:
def load_json_file(path):
    with open(path, 'r') as f:
        return json.load(f)

In [ ]:
def write_json_file(path, data):
    with open(path, 'w') as f:
        json.dump(data, f)
    return

In [ ]:
# def check_make_directory(path):
#     if not os.path.exists(path):
#         os.makedirs(path)
#         print('Created directory: {}'.format(path))
#     else:
#         print('Directory {} already exists!'.format(path))

In [ ]:
class DataLoader(object):
    '''
    Loads data from file paths stored in Config() class
    '''
    def __init__(self):
        super(DataLoader, self).__init__()
        self.config = Config()
        self.load_data()
        
    def load_data(self):
        path_to_data_files = file_paths
        self.raw_data = load_json_file(path_to_data_files['path_to_raw_data'])
        self.target_vocab = load_json_file(path_to_data_files['path_to_target_vocab'])
        self.char_vocab = load_json_file(path_to_data_files['path_to_char_vocab'])
        self.pos_vocab = load_json_file((path_to_data_files['path_to_pos_vocab']))
        self.glove_feat_matrix = np.load(path_to_data_files['path_to_glove_embed'])
        #self.xlnet_vocab = load_json_file(path_to_data_files['path_to_xlnet_vocab'])
        #self.xlnet_indices = load_json_file(path_to_data_files['path_to_xlnet_indices'])
        

In [ ]:
class DataHandler(object):
    '''
    1. calls DataLoader to load the data
    2. data is split into train, valid and test according to the mode specified in configuration
    3. conf_update() updates the configuration after data is split
    4. collate_fn creates batches according to batch size.
    '''
    def __init__(self):
        super(DataHandler, self).__init__()
        self.config = Config()
        self.init_generators()
        self.conf_update()
        
    def init_generators(self):
        self.data = DataLoader()
        if self.config.MODE == 'train':
            self.train_dataset = Dataset(self.data.raw_data['train'])
            self.trainset_generator = torch_data.DataLoader(
                self.train_dataset, batch_size = self.config.BATCH_SIZE,
                collate_fn  = self.collate_fn, shuffle=True, num_workers = self.config.NUM_WORKER,
                drop_last = True
            )
            
            self.valid_dataset = Dataset(self.data.raw_data['valid'])
            self.validset_generator = torch_data.DataLoader(
                self.valid_dataset, batch_size = self.config.BATCH_SIZE,
                collate_fn  = self.collate_fn, shuffle=False, num_workers = self.config.NUM_WORKER,
                drop_last = False
            )
        
        else:
            self.test_dataset = Dataset(self.data.raw_data['valid'])
            self.testset_generator = torch_data.DataLoader(
                self.test_dataset, batch_size = self.config.BATCH_SIZE,
                collate_fn  = self.collate_fn, shuffle=False, num_workers = self.config.NUM_WORKER,
                drop_last = False
            )
            
    def conf_update(self):
        def get_batch_size(dataset_size):
            if dataset_size % self.config.BATCH_SIZE == 0:
                return dataset_size // self.config.BATCH_SIZE
            else:
                return dataset_size // self.config.BATCH_SIZE + 1
            
        # data parameters
        self.config.TGT_VOCAB_SIZE = len(self.data.target_vocab.keys()) # 4
        self.config.NUM_CLASSES = self.config.TGT_VOCAB_SIZE       # 4
        self.config.CHAR_VOCAB_SIZE = len(self.data.char_vocab.keys())  # contains latin characters (119)
        self.config.POS_VOCAB_SIZE = len(self.data.pos_vocab.keys())    # 47
        self.config.START_IDX = self.data.target_vocab[self.config.START_SYMBOL] # 0
        self.config.END_IDX = self.data.target_vocab[self.config.END_SYMBOL]     # 1
        self.config.PAD_IDX = self.data.target_vocab[self.config.PAD_SYMBOL]     # 2

        
        
        # Training Parameters
        if self.config.MODE == 'train':
            self.config.train_size = len(self.train_dataset) # 32162
            self.config.valid_size = len(self.valid_dataset) # 4030
            print('Training dataset size: {}'.format(self.config.train_size))
            print('Validation dataset size: {}'.format(self.config.valid_size))
            self.config.num_batch_train = get_batch_size(self.config.train_size)
            self.config.num_batch_valid = get_batch_size(self.config.valid_size)
        else:
            self.config.test_size = len(self.test_dataset)
            print('Testing dataset size: {}'.format(self.config.test_size))
            self.config.num_batch_test = get_batch_size(self.config.test_size)
            
    
    def collate_fn(self,data):
        
        # Step 1: Unpack the sequences
        #print(data)
        df_bert, df_glove, df_char, df_pos, ys, labels, df_xlnet = zip(*data)
        df_bert, df_glove, df_char, df_pos, ys, labels, df_xlnet = list(df_bert), list(df_glove), list(df_char), list(df_pos), list(ys), list(labels), list(df_xlnet)
        batch_size = len(labels)
        
        # Data pre-processing
        #  Step 2: convert lists to tensors
        # print(df_glove[:20])
        df_glove = [self.data.glove_feat_matrix[df_glove[i]] for i in range(batch_size)]
        df_glove = [torch.Tensor(seq) for seq in df_glove]
        df_glove = pad_sequence(df_glove, batch_first=True, padding_value=self.config.PAD_IDX)

        df_bert = [torch.Tensor(seq) for seq in df_bert]
        df_xlnet = [torch.Tensor(seq) for seq in df_xlnet]
        df_pos = [torch.Tensor(seq) for seq in df_pos]
        df_char = [torch.Tensor(item) for sublist in df_char for item in sublist]

        ys = [torch.Tensor(seq) for seq in ys]
        
        # Step 3: create pad sequence
        df_bert = pad_sequence(df_bert, batch_first=True, padding_value=self.config.PAD_IDX)
        df_xlnet = pad_sequence(df_xlnet, batch_first=True, padding_value=self.config.PAD_IDX)
        df_char = pad_sequence(df_char, batch_first=True, padding_value=self.config.PAD_IDX)
        df_char = [df_char[i] for i in range(df_char.shape[0])]
        df_pos = pad_sequence(df_pos, batch_first=True, padding_value=self.config.PAD_IDX)

        ys = pad_sequence(ys, batch_first=True, padding_value=self.config.PAD_IDX)
        
        # Step 4: record source seq lengths
        attn_mask = df_bert != self.config.PAD_IDX
        df_bert_lens = attn_mask.sum(-1)
        mask = df_pos != self.config.PAD_IDX
        df_glove_lens = mask.sum(-1)
        df_char = iter(df_char)
        df_char = [list(islice(df_char, elem))
                   for elem in df_glove_lens.cpu().detach().numpy().tolist()]
        df_char = [torch.vstack(seq) for seq in df_char]
        df_char = pad_sequence(df_char, batch_first=True, padding_value=self.config.PAD_IDX)
        # print("Bert attention mask:\n")
        # print(attn_mask[:20])
        # print("shape\n")
        # print(attn_mask.size())
        # xlnet_attn_mask = torch.Tensor([0 for x in df_xlnet])
        # print(df_xlnet[:20])
        # print("df_bert is \n")
        # print(df_bert[:20])
        #xlnet_attn_mask = torch.Tensor([x for x in df_xlnet])#df_xlnet != self.config.PAD_IDX
        xlnet_attn_mask = list()
        for row in df_xlnet:
            xlnet_attn_mask.append(list())
            for val in row:
                if val:
                    xlnet_attn_mask[-1].append(1)
                else:
                    xlnet_attn_mask[-1].append(0)
        xlnet_attn_mask = torch.Tensor(xlnet_attn_mask)
        df_xlnet_lens = xlnet_attn_mask.sum(-1)



        return {'df_bert': df_bert.long().to(self.config.DEVICE),
                'df_xlnet': df_xlnet.long().to(self.config.DEVICE),
                'df_glove': df_glove.to(self.config.DEVICE),
                'df_char': df_char.long().to(self.config.DEVICE),
                'df_pos': df_pos.long().to(self.config.DEVICE),
                'ys': ys.long().to(self.config.DEVICE),
                'df_bert_lens': df_bert_lens.long().to(self.config.DEVICE),
                'df_bert_attn_mask': attn_mask.long().to(self.config.DEVICE),
                'df_glove_lens': df_glove_lens,
                'labels': labels,
                'df_xlnet_lens':df_xlnet_lens,
                'df_xlnet_attn_mask':xlnet_attn_mask}

# Embedding

### 1. Glove Embedding

In [ ]:
class GloveEmbeddingGenerator(nn.Module):
    def __init__(self, config, path):
        super(GloveEmbeddingGenerator, self).__init__()
        self.config = config
        print('Loading Pre-trained Glove Embeddings')
        embed_weights = np.load(path)
        vocab_size, dim = embed_weights.shape
        embed_weights = torch.FloatTensor(embed_weights)
        self.embedding_model = nn.Embedding(vocab_size, dim, padding_idx=config.PAD_IDX)
        self.embedding_model.weight = nn.Parameter(embed_weights)

    def forward(self, df):
        # [batch_size, max_seq_len, hidden_dim]
        df = self.embedding_model(df)
        return df

### 2. Character Embedding

In [ ]:
class CharacterEmbedding(nn.Module):
    '''
    We are assigning numbers stored in character json file to characters 
     
    Inupt : (input length, sentence length, word length)
    Output: (input length, sentence length, character embedding size)
    '''
    def __init__(self, config):
        super(CharacterEmbedding, self).__init__()
        self.config = config
        self.embd_size = config.CHAR_EMBED_DIM
        self.embedding = nn.Embedding(config.CHAR_VOCAB_SIZE, config.CHAR_EMBED_DIM, padding_idx=config.PAD_IDX)
        self.conv = nn.ModuleList([nn.Conv2d(1, config.CHAR_EMBED_CNN_NUM_OUT_CHANNELS,
                                             (f[0], f[1])) for f in config.CHAR_EMBED_CHAR_FILTERS])
        self.dropout = nn.Dropout(config.CHAR_EMBED_DROPOUT_RATE)

    def forward(self, x):
        input_shape = x.size()
        word_len = x.size(2)
        x = x.view(-1, word_len)
        x = self.embedding(x)
        x = x.view(*input_shape, -1)
        x = x.sum(2)

        # CNN
        x = x.unsqueeze(1)
        x = [F.relu(conv(x)) for conv in self.conv]
        x = [xx.view((xx.size(0), xx.size(2), xx.size(3), xx.size(1))) for xx in x]
        x = [torch.sum(xx, 2) for xx in x]
        x = torch.cat(x, 1)
        x = self.dropout(x)

        return x


### 3. POS Embedding

In [ ]:
class POSEmbeddingGenerator(nn.Module):
    '''
    We are assigning numbers stored in POS json file to POS tags 
    '''
    def __init__(self, config):
        super(POSEmbeddingGenerator, self).__init__()
        self.embedding_model = nn.Embedding(config.POS_VOCAB_SIZE, config.POS_EMBED_DIM, padding_idx=config.PAD_IDX)
        self.embedding_model.to(config.DEVICE)

    def forward(self, df):
        df = self.embedding_model(df)
        # [batch_size, max_seq_len, hidden_dim]
        return df

### 4. BERT Embedding

In [ ]:
class BERTEembeddingGenerator(nn.Module):
    """
    Embedding with pretrained bert-base-uncased Model
    """
    def __init__(self, config):
        super(BERTEembeddingGenerator, self).__init__()
        self.embedding_model = BertModel.from_pretrained(
            config.PRETRAINED_BERT_MODEL_NAME,
            return_dict=True
        )
        self.embedding_model.to(config.DEVICE)

    def forward(self, df, attn_mask):
        df = self.embedding_model(df, attention_mask=attn_mask)
        df = df.last_hidden_state
        return df

### 5. XLNet Embedding

In [ ]:
class XLNETEmbeddingGenerator(nn.Module):
  """
    Embedding with pretrained xlnet-large-cased model
  """
  def __init__(self, config):
    super(XLNETEmbeddingGenerator, self).__init__()
    self.embedding_model = XLNetModel.from_pretrained(
        config.PRETRAINED_XLNET_MODEL_NAME, 
        return_dict=True
    )
    self.config = config
    self.embedding_model.to(config.DEVICE)

  def forward(self, df, xlnet_attn_mask):
    df = self.embedding_model(df, attention_mask=xlnet_attn_mask.to(self.config.DEVICE))
    df = df.last_hidden_state

    return df

## Highway Network

In [ ]:
class HighwayNetwork(nn.Module):
    def __init__(self, config):
        super(HighwayNetwork, self).__init__()
        self.config = config
        self.n_layers = config.HIGHWAY_NUM_LAYERS
        in_size = config.CHAR_EMBED_CNN_NUM_OUT_CHANNELS + config.PRETRAINED_GLOVE_MODEL_EMBED_DIM
        self.normal_layer = nn.ModuleList([nn.Linear(in_size, in_size) for _ in range(self.n_layers)])
        self.gate_layer = nn.ModuleList([nn.Linear(in_size, in_size) for _ in range(self.n_layers)])

    def forward(self, x):
        for i in range(self.n_layers):
            normal_layer_ret = F.relu(self.normal_layer[i](x))
            gate = torch.sigmoid(self.gate_layer[i](x))

            x = gate * normal_layer_ret + (1 - gate) * x

        return x

### Attention Flow

In [ ]:
class POSLayerContextualEmbedding(nn.Module):
    """
    Contextual POS Embedding using Bi-LSTM
    """
    def __init__(self, config):
        super(POSLayerContextualEmbedding, self).__init__()

        self.context_LSTM = nn.LSTM(input_size=config.POS_EMBED_DIM,
                                 hidden_size=config.LSTM_HIDDEN_DIM,
                                 bidirectional=True,
                                 batch_first=True,
                                 dropout=config.LSTM_DROP_RATE)
        self.dropout = nn.Dropout(config.LSTM_DROP_RATE)

    def forward(self, x):
        x = self.dropout(x)
        x, _ = self.context_LSTM(x)
        return x


### Contextual Embedding Layer

In [ ]:
class ContextualEmbeddingLayer(nn.Module):
    """
        Contextual Embedding layer
    """
    def __init__(self, config):
        super(ContextualEmbeddingLayer, self).__init__()

        self.context_LSTM = nn.LSTM(input_size=config.EMBEDDING_DIM,
                                 hidden_size=config.LSTM_HIDDEN_DIM,
                                 bidirectional=True,
                                 batch_first=True,
                                 dropout=config.LSTM_DROP_RATE)
        self.dropout = nn.Dropout(config.LSTM_DROP_RATE)

    def forward(self, x):
        x = self.dropout(x)
        x, _ = self.context_LSTM(x)
        return x

### Attentation Flow Layer

In [ ]:
class AttentionFlowLayer(nn.Module):
    """
    
    """
    def __init__(self, config):
        super(AttentionFlowLayer, self).__init__()

        self.config = config

        self.W = nn.Linear(6*config.LSTM_HIDDEN_DIM, 1, bias=False)

    def forward(self, emb_context, emb_query):
        max_len_context, max_len_query = emb_context.size(1), emb_query.size(1)

        shape = (emb_context.shape[0], max_len_context,
                 max_len_query, self.config.LSTM_HIDDEN_DIM*2)  

        emb_context_ex = emb_context.unsqueeze(2)          
        emb_context_ex = emb_context_ex.expand(shape)  
        emb_query_ex = emb_query.unsqueeze(1)               
        emb_query_ex = emb_query_ex.expand(shape)      

        cat_data = torch.cat((emb_context_ex,
                              emb_query_ex,
                              torch.mul(emb_context_ex, emb_query_ex)), 3)  
        S = self.W(cat_data).view(emb_context.shape[0],
                                  max_len_context,
                                  max_len_query)  

        c2q_attn = torch.bmm(F.softmax(S, dim=-1), emb_query) 

        
        b = F.softmax(torch.max(S, 2)[0], dim=-1)  
        q2c_attn = torch.bmm(b.unsqueeze(1), emb_context) 
        q2c_attn = q2c_attn.repeat(1, max_len_context, 1)  

        G = torch.cat((emb_context,
                       c2q_attn,
                       emb_context.mul(c2q_attn),
                       emb_context.mul(q2c_attn)),
                      2)  
        if self.config.OUTPUT_ATTN:
            return G, S

        return G


### Modeling Layer

In [ ]:
class ModelingLayer(nn.Module):
    """
       Modeling layer
    """
    def __init__(self, config):
        super(ModelingLayer, self).__init__()

        self.modeling_LSTM1 = nn.LSTM(input_size=config.LSTM_HIDDEN_DIM * 8,
                                   hidden_size=config.LSTM_HIDDEN_DIM,
                                   bidirectional=True,
                                   batch_first=True,
                                   dropout=config.LSTM_DROP_RATE)
        self.dropout = nn.Dropout(config.LSTM_DROP_RATE)

    def forward(self, x, seq_context_lens):
        x = self.dropout(x)
        x, _ = self.modeling_LSTM1(x)
        return x

## Reading Comprehension

In [ ]:
class ReadingComprehensionDetector(nn.Module):
    def __init__(self, config):
        super(ReadingComprehensionDetector, self).__init__()
        self.config = config
        
        # Embedding layers
        # ------------------------------------------------------
        
        # BERT embedding layer
        self.bert_embedding_layer = BERTEembeddingGenerator(config)
        self.bert_embedding_transform_layer = nn.Linear(config.PRETRAINED_BERT_MODEL_EMBED_DIM,
                                                        config.PRETRAINED_GLOVE_MODEL_EMBED_DIM + config.CHAR_EMBED_DIM )
        
        # XLNet embedding layer
        self.xlnet_embedding_layer = XLNETEmbeddingGenerator(config)##
        self.xlnet_embedding_transform_layer = nn.Linear(config.PRETRAINED_XLNET_MODEL_EMBED_DIM,
                                                         config.PRETRAINED_GLOVE_MODEL_EMBED_DIM + config.CHAR_EMBED_DIM)##
        
        # character embedding layer
        self.char_embedding_layer = CharacterEmbedding(config)
        
        # POS embedding layer
        self.pos_embedding_layer = POSEmbeddingGenerator(config)
        
        # Highway network
        self.highway_network = HighwayNetwork(config)

        # # Contextual Embedding layer
        self.contextual_embedding_layer = ContextualEmbeddingLayer(config)
        self.pos_layer_contextual_embedding = POSLayerContextualEmbedding(config)

        # Attention flow layer
        # ------------------------------------------------------
        self.attention_flow_layer_0 = AttentionFlowLayer(config)
        self.attention_flow_layer_1 = AttentionFlowLayer(config)
        self.attention_flow_layer_2 = AttentionFlowLayer(config)##


        # Modeling layer
        # ------------------------------------------------------
        self.modeling_layer_0 = ModelingLayer(config)
        self.modeling_layer_1 = ModelingLayer(config)
        self.modeling_layer_2 = ModelingLayer(config)##

        # Output layer
        # ------------------------------------------------------
        self.output_layer = nn.Linear(config.LSTM_HIDDEN_DIM * 2, config.NUM_CLASSES)

    def forward(self, data):
        
        ## Tensor data is stored in respective variables
        bert_token_seq = data['df_bert']
        xlnet_token_seq = data['df_xlnet']
        glove_token_seq = data['df_glove']
        pos_token_seq = data['df_pos']
        char_seq = data['df_char']
        seq_context_lens = data['df_bert_lens']
        seq_query_lens = data['df_glove_lens']
        bert_attn_mask = data['df_bert_attn_mask']
        xlnet_attn_mask = data['df_xlnet_attn_mask']
        xlnet_seq_context_lens = data['df_xlnet_lens']
        # print(type(bert_attn_mask), bert_attn_mask.size())
        # print(type(xlnet_attn_mask), xlnet_attn_mask.size())
        # print(xlnet_token_seq.size())

        # 1. Embedding Layers
        
        context_seq = self.bert_embedding_layer(bert_token_seq, bert_attn_mask)
        context_seq = self.bert_embedding_transform_layer(context_seq)
        del bert_token_seq

        xlnet_context_seq = self.xlnet_embedding_layer(xlnet_token_seq, xlnet_attn_mask)
        xlnet_context_seq = self.xlnet_embedding_transform_layer(xlnet_context_seq)
        del xlnet_token_seq
        
        pos_token_seq = self.pos_embedding_layer(pos_token_seq)
        char_seq = self.char_embedding_layer(char_seq)
        query_seq = torch.cat((char_seq, glove_token_seq), -1)
        query_seq = self.highway_network(query_seq)
        del glove_token_seq, char_seq

        # 2. Contextual Embedding Layer
        context_seq = self.contextual_embedding_layer(context_seq)
        xlnet_context_seq = self.contextual_embedding_layer(xlnet_context_seq)  
        query_seq = self.contextual_embedding_layer(query_seq)  
        pos_token_seq = self.pos_layer_contextual_embedding(pos_token_seq)
        if self.config.OUTPUT_ATTN:
            G_0, S_0 = self.attention_flow_layer_0(query_seq, pos_token_seq)
            G_2, S_2 = self.attention_flow_layer_2(context_seq, xlnet_context_seq)
        else:
            G_0 = self.attention_flow_layer_0(query_seq, pos_token_seq)
            G_2 = self.attention_flow_layer_2(context_seq, xlnet_context_seq)
        M_0 = self.modeling_layer_0(G_0, seq_query_lens)
        M_2 = self.modeling_layer_2(G_2, seq_context_lens)

        # 3. Attention Flow Layer
        
        if self.config.OUTPUT_ATTN:
            G_1, S_1 = self.attention_flow_layer_1(M_2, M_0)
            #G_3, S_3 = self.attention_flow_layer_2(xlnet_context_seq, M_2)
        else:
            G_1 = self.attention_flow_layer_1(M_2, M_0)  
            #G_3 = self.attention_flow_layer_2(xlnet_context_seq, M_2)

        # 4. Modeling Layer
        M_1 = self.modeling_layer_1(G_1, G_2)  

        # 5. Final Linear Layer (Seq2seq Bilstm with attention)
       
        out = self.output_layer(M_1)
        if self.config.OUTPUT_ATTN:
            return F.log_softmax(out, dim=-1), S_1

        return F.log_softmax(out, dim=-1)

## Model Evaluation

In [ ]:
def load_init_model(model_module, config):
    """
    Initialize a model and load a checkpoint if so desired (if the checkpoint is available.)
    """

    epoch_start = 0

    model = model_module(config).to(config.DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
    save_path = config.PATH_TO_CHECKPOINT.format(config.MODEL_NAME, config.LOAD_CHECKPOINT_TYPE)
    if os.path.exists(save_path) and config.CONTINUE_TRAIN:
        print('Loading previous model from {}'.format(save_path))
        model, optimizer, epoch_start = load_checkpoint(model, optimizer, save_path, config.DEVICE)
    else:
        print('=> ERROR : No checkpoint found!!\nTraining from scratch!')
    model.eval()

    return model, optimizer, epoch_start

In [ ]:
def count_parameters(model):
    # get total size of trainable parameters
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def load_checkpoint(model, optimizer, filename='checkpoint.pth.tar', device='cpu'):
    '''
        Load previous model from a checkpoint.

    '''
    start_epoch = 0
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        if device == 'cpu':
            checkpoint = torch.load(filename, map_location=lambda storage, location: storage)
        else:
            checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}'"
              .format(filename))
    else:
        print("=> ERROR : No checkpoint found at '{}'".format(filename))

    return model, optimizer, start_epoch

In [ ]:
def save_model(save_path, model, optimizer, epoch):
    """
    Creates a checkpoint
    """
    state = {'epoch': epoch + 1,
             'state_dict': model.state_dict(),
             'optimizer': optimizer.state_dict()}

    torch.save(state, save_path)

In [ ]:
def load_model_from_checkpoint(model_module, config):
    """
    Initialize a model and load a checkpoint.
    """

    model = model_module(config).to(config.DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
    save_path = config.PATH_TO_CHECKPOINT.format(config.MODEL_NAME, config.LOAD_CHECKPOINT_TYPE)
    if os.path.exists(save_path):
        print('Loading previous model from {}'.format(save_path))
        model, _, _ = load_checkpoint(model, optimizer, save_path, config.DEVICE)
    else:
        raise NotImplementedError('=> ERROR : Checkpoint does not exist !!')
    model.eval()

    return model


## Evaluation

In [ ]:
def compute_performance(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

In [ ]:
def remove_index(seq, index):
    return [i for i in seq if i != index]

In [ ]:
def post_process(srcs, tgts, preds, config):
    # remove pad index
    srcs = [remove_index(src, config.PAD_IDX) for src in srcs]
    tgts = [remove_index(tgt, config.PAD_IDX) for tgt in tgts]
    preds = [remove_index(pred, config.PAD_IDX) for pred in preds]

    # remove end index
    end_indices = [p.index(config.END_IDX)+1 if config.END_IDX in p else len(p) for p in preds]
    preds = [p[:index] for index, p in zip(end_indices, preds)]
    return srcs, tgts, preds

In [ ]:
def post_process_eval(srcs, tgts, preds, config):
    # remove pad index
    srcs = [[p for p in src] for src in srcs]
    tgts = [remove_index(tgt, config.PAD_IDX) for tgt in tgts]
    preds = [[p for p in pred] for pred in preds]

    # remove end index
    end_indices = [len(p) for p in tgts]
    preds = [p[:index] for index, p in zip(end_indices, preds)]
    return srcs, tgts, preds

In [ ]:
class MeasureAverage(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
# def rand_sample(srcs, tars, preds):
#     src, tar, pred = random.choice([(src, tar, pred) for src, tar, pred in zip(srcs, tars, preds)])
#     src = [str(s) for s in src]
#     tar = [str(s) for s in tar]
#     pred = [str(s) for s in pred]

#     return ' '.join(src), ' '.join(tar), ' '.join(pred)

In [ ]:
def check_seq(tar, pred):
    min_len = min([len(tar), len(pred)])
    if sum(np.equal(tar[:min_len], pred[:min_len])) == len(tar):
        return 1
    return 0

In [ ]:
def check_cls(tar, pred):
    tar_cls = 1 if 4 in tar else 0
    pred_cls = 1 if 4 in pred else 0
    return tar_cls, pred_cls

In [ ]:
def get_seq_acc(tars, preds):
    size = len(tars)
    a = 0
    for i in range(size):
        tar = tars[i]
        pred = preds[i]
        a += check_seq(tar, pred)

    return np.float32(a/size)

In [ ]:
def get_seq_acc_test(tars, preds):
    size = len(tars)
    a = 0
    pred_seq = []
    for i in range(size):
        tar = tars[i]
        pred = preds[i]
        correctness = check_seq(tar, pred)
        a += correctness
        pred_seq.append(correctness)

    return np.float32(a/size), pred_seq

In [ ]:
def get_cls_acc_test(tars, preds):
    size = len(tars)
    cur_tars, cur_preds = [], []
    for i in range(size):
        tar = tars[i]
        pred = preds[i]
        t, p = check_cls(tar, pred)
        cur_tars.append(t)
        cur_preds.append(p)

    return cur_tars, cur_preds

In [ ]:
def get_seq_performance(tars, preds):
    size = len(tars)
    tars_tok_seq = []
    pred_tok_seq = []

    for i in range(size):
        tar = tars[i]
        pred = preds[i]
        tars_tok_seq += tar
        pred_tok_seq += pred

    tars_tok_seq = [item for sublist in tars_tok_seq for item in sublist]
    pred_tok_seq = [item for sublist in pred_tok_seq for item in sublist]

    tars_tok_seq = [1 if t == 4 else 0 for t in tars_tok_seq]
    pred_tok_seq = [1 if t == 4 else 0 for t in pred_tok_seq]
    token_accuracy = accuracy_score(tars_tok_seq, pred_tok_seq)
    token_precision = precision_score(tars_tok_seq, pred_tok_seq)
    token_recall = recall_score(tars_tok_seq, pred_tok_seq)
    token_f1 = f1_score(tars_tok_seq, pred_tok_seq)

    return {
        'token_accuracy': token_accuracy,
        'token_precision': token_precision,
        'token_recall': token_recall,
        'token_f1': token_f1
    }

In [ ]:
def get_cls_performance(tars, preds):
    token_accuracy = accuracy_score(tars, preds)
    token_precision = precision_score(tars, preds)
    token_recall = recall_score(tars, preds)
    token_f1 = f1_score(tars, preds)

    return {
        'cls_accuracy': token_accuracy,
        'cls_precision': token_precision,
        'cls_recall': token_recall,
        'cls_f1': token_f1
    }

In [ ]:
def train_step(model, optimizer, data_handler, criterion, epoch, writer):
    """
    Trains model for one epoch
    """
    model.train()
    
    loss_epoch = MeasureAverage()
    seq_acc_epoch = MeasureAverage()

    # train data for a single epoch
    bbar = tqdm(enumerate(data_handler.trainset_generator), ncols=100, leave=False)
    for idx, data in bbar:
        torch.cuda.empty_cache()
        batch_size = data['df_bert'].shape[0]

        # model is forward passed to compute the node embeddings
        ys_ = model(data)
        loss = criterion(ys_.reshape(-1, data_handler.config.TGT_VOCAB_SIZE), data['ys'].reshape(-1))

        # computes negative sampling loss and updates the model weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # loss evaluation metrics
        loss = loss.detach().cpu().item()
        loss_epoch.update(loss, batch_size)

        # evaluation results
        df = data['df_bert'].cpu().detach().numpy()
        ys = data['ys'].cpu().detach().numpy()
        ys_ = torch.argmax(ys_, dim=2).cpu().detach().numpy()
        df, ys, ys_ = post_process_eval(df, ys, ys_, data_handler.config)
        seq_acc = get_seq_acc(ys, ys_)
        seq_acc_epoch.update(seq_acc, 1)
        
        # random sample to show
        if data_handler.config.VERBOSE and idx % data_handler.config.DISPLAY_FREQ == 0:
            src, tar, pred = rand_sample(df, ys, ys_)
            bbar.set_description("Phase: [Train] | Batch: {}/{} | Train Loss: {:.3f} | Seq Acc: {:.3f}\n src: {}\n tgt: {}\n pred: {}\n".format(idx,
                                                                                             data_handler.config.num_batch_train,
                                                                                             loss, seq_acc,
                                                                                             src, tar, pred))
        # set display bar
        else:
            bbar.set_description("Phase: [Train] | Batch: {}/{} | Train Loss: {:.5f} | Seq Acc: {:.3f}".format(idx,
                                                                                             data_handler.config.num_batch_train,
                                                                                             loss, seq_acc))
        if idx % data_handler.config.SAVE_FREQ == 0:
            if data_handler.config.USE_TENSORBOARD:
                writer.add_scalar('train_loss', loss_epoch.avg, epoch*data_handler.config.num_batch_train+idx)
                writer.add_scalar('train_seq_acc', seq_acc, epoch*data_handler.config.num_batch_train+idx)

    return loss_epoch.avg, seq_acc_epoch.avg


In [ ]:
def valid_step(model, data_handler, criterion):
    """
    Valid model for one epoch
    """
    
    model.eval()
    torch.cuda.empty_cache()
    # performance recorders
    loss_epoch = MeasureAverage()
    seq_acc_epoch = MeasureAverage()

    # valid for a single epoch
    bbar = tqdm(enumerate(data_handler.validset_generator), ncols=100, leave=False)
    for idx, data in bbar:
        torch.cuda.empty_cache()
        batch_size = data['df_bert'].shape[0]

        # model is forward passed
        with torch.no_grad():
            # model forward passed to compute loss
            ys_ = model(data)
            loss = criterion(ys_.reshape(-1, data_handler.config.TGT_VOCAB_SIZE), data['ys'].reshape(-1))

        # loss evaluation metrics
        loss = loss.detach().cpu().item()
        loss_epoch.update(loss, batch_size)

        # evaluation results
        df = data['df_bert'].cpu().detach().numpy()
        ys = data['ys'].cpu().detach().numpy()
        ys_ = torch.argmax(ys_, dim=2).cpu().detach().numpy()
        df, ys, ys_ = post_process_eval(df, ys, ys_, data_handler.config)
        seq_acc = get_seq_acc(ys, ys_)
        seq_acc_epoch.update(seq_acc, batch_size)

        # random sample to show
        if data_handler.config.VERBOSE:
            src, tar, pred = rand_sample(df, ys, ys_)
            bbar.set_description("Phase: [Valid] | Batch: {}/{} | Valid Loss: {:.3f} | Seq Acc: {:.3f}\n src: {}\n tgt: {}\n pred: {}\n".format(idx,
                                                                                             data_handler.config.num_batch_valid,
                                                                                             loss,
                                                                                             seq_acc,
                                                                                             src, tar, pred))

        # set display bar
        else:
            bbar.set_description("Phase: [Valid] | Batch: {}/{} | Valid Loss: {:.3f} | Seq Acc: {:.3f}".format(idx,
                                                                                          data_handler.config.num_batch_valid,
                                                                                          loss,
                                                                                          seq_acc))
    return loss_epoch.avg, seq_acc_epoch.avg

## Training Model

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [ ]:
def train():
    """
    Train function for Idiom Detection Model
    """
    
    # Initializing Data Loader
    data_handler = DataHandler()

    # Initializing the model
    save_path = Config.PATH_TO_CHECKPOINT
    detector_model, optimizer, epoch_start = load_init_model(ReadingComprehensionDetector, data_handler.config)
    
    # Freezing the pre-trained embedding layer
    for param in detector_model.bert_embedding_layer.embedding_model.base_model.parameters():
        param.requires_grad = False
    
    for param in detector_model.xlnet_embedding_layer.embedding_model.base_model.parameters():
        param.requires_grad = False

    # Adding the loss function "negative log likelihood loss"
    # It’s a cost function that tells us how bad it’s performing. The lower, the better.
    criterion = torch.nn.NLLLoss(ignore_index=data_handler.config.PAD_IDX)
    
    # Adding Tensorboard writer
    writer = None
    if Config.USE_TENSORBOARD:
        writer = SummaryWriter(log_dir='./runs/{}_{}'.format(Config.MODEL_NAME, datetime.today().strftime('%Y-%m-%d')))
    
    # Initializing best valid accuracy to -Infinity
    best_valid_acc = float('-inf')

    # Training the model

    ebar = trange(epoch_start, Config.NUM_EPOCHS, desc='EPOCH', ncols=130, leave=True)
    set_seed(Config.SEED)
    for epoch in ebar:
        # Training
        _, _ = train_step(detector_model, optimizer, data_handler, criterion, epoch, writer)

        # Validation
        if epoch % Config.VALID_FREQ == 0:
            valid_loss, valid_seq_acc = valid_step(detector_model, data_handler, criterion)
            if best_valid_acc < valid_seq_acc:
                best_valid_acc = valid_seq_acc
                # save the best model seen so far
                save_model(save_path.format(Config.MODEL_NAME, 'best'), detector_model, optimizer, epoch)
            if Config.USE_TENSORBOARD:
                writer.add_scalar('valid_loss', valid_loss, epoch)
                writer.add_scalar('valid_seq_acc', valid_seq_acc, epoch)

        # save the latest model
        save_model(save_path.format(Config.MODEL_NAME, 'latest'), detector_model, optimizer, epoch)

    return


### Running train function

In [ ]:
train()

In [ ]:
!zip -r ./checkpoints.zip ./checkpoints

updating: checkpoints/ (stored 0%)
updating: checkpoints/IdiomDetector_magpie_random_best.mdl (deflated 7%)
updating: checkpoints/IdiomDetector_magpie_random_latest.mdl (deflated 7%)


In [ ]:
from google.colab import files
files.download("./checkpoints.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Testing Model

In [ ]:
def test_step(model, data_handler):
    """
    Evaluating model for one epoch
    """
    model.eval()
    torch.cuda.empty_cache()
    # performance recorders
    seq_acc_epoch = MeasureAverage()
    cls_tars, cls_preds = [], []
    target_seqs = []
    predict_seqs = []
    # valid for a single epoch
    bbar = tqdm(enumerate(data_handler.testset_generator), ncols=100, leave=True)
    for idx, data in bbar:
        
        # model is forward passed
        with torch.no_grad():
            # model forward pass to compute loss
            ys_ = model(data)
        # evaluate results
        batch_size = data['xs_bert'].shape[0]
        xs = data['xs_bert'].cpu().detach().numpy()  
        ys = data['ys'].cpu().detach().numpy()  

        ys_ = torch.argmax(ys_, dim=2).cpu().detach().numpy()  
        xs, ys, ys_ = post_process_eval(xs, ys, ys_, data_handler.config)
        target_seqs.append(ys)
        predict_seqs.append(ys_)
        seq_acc, correctness = get_seq_acc_test(ys, ys_)
        cur_cls_tars, cur_cls_preds = get_cls_acc_test(ys, ys_)
        cls_tars += cur_cls_tars
        cls_preds += cur_cls_preds
        seq_acc_epoch.update(seq_acc, batch_size)

        # format data
        for i in range(len(ys_)):
            ys_[i] = [s.item() for s in ys_[i]]

        # show progress
        bbar.set_description("Phase: [Test] | Batch: {}/{} | Seq Acc: {:.3f}".format(idx,
                                                                                      data_handler.config.num_batch_test,
                                                                                    seq_acc))
    performance_tok = get_seq_performance(target_seqs, predict_seqs)
    performance_tok['seq_acc'] = seq_acc_epoch.avg
    performance_cls = get_cls_performance(cls_tars, cls_preds)

    return performance_tok, performance_cls

In [ ]:
def inference():
    """
    The inference script for the DISC model.
    """
    # Initialize a data loader
    
    data_handler = DataHandler()

    # Manage and initialize model
    
    # Initialize model
    detector_model, optimizer, epoch_start = load_init_model(DetectorMdl, data_handler.config)

    # Run model inference
    
    performance_token, performance_cls = test_step(detector_model, data_handler)
    print('Performance at token and seq level: ')
    print(performance_token)
    print('--------------------------------------------------------------')
    print('Performance for binary classification: ')
    print(performance_cls)

    return

In [ ]:
!ls ./checkpoints

In [ ]:
!zip checkpoints_v1.zip ./checkpoints/*

In [ ]:
!ls ./

In [ ]:
!rm ./checkpoints_v1.zip

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink(r'checkpoints_v1.zip')

### Testing

In [ ]:
raw_data = load_json_file("E:\BTech\Sem-8\Project\DISC-master\data\MAGPIE\processed\MAGPIE_random_read_comp_data.json")

NameError: ignored

In [ ]:
len(raw_data['train']), len(raw_data['valid']), len(raw_data['test'])

NameError: ignored

In [ ]:
#raw_data